## DID估计量

### DID案例

1980
年，肯塔基州提高了工人赔偿所涵盖的每周收入上限。我们想知道这项新政策是否导致工人失业时间更长。如果福利不够丰厚，那么工人可能会就工伤起诉公司，而过于丰厚的福利可能会引发道德风险问题，并促使工人在工作中更加鲁莽，或者声称非工作时间的受伤是在工作时发生的。

我们关心的主要结果变量是log_duration，即工人赔偿福利的持续时间（以周为单位）的对数。我们对其取对数是因为这个变量相当偏斜
——
大多数人失业几周，而有些人失业很长时间。该政策的设计使得上限的提高不会影响低收入工人，但会影响高收入工人，所以我们将低收入工人作为对照组，将高收入工人作为处理组。

主要变量：

- duration：失业救济金的持续时间，以周为单位测量。
- log_duration：duration的对数版本，即 log(duration)。
- after_1980：一个指示变量，如果观测值发生在 1980 年政策变化之前则为
  0，发生在之后则为 1。这是我们的时间变量（或者说是前后变量）。
- highearn：一个指示变量，如果观测值对应的是低收入者则为 0，高收入者则为
  1。这是我们的分组变量（或者说是处理组 / 对照组变量）。

### 预处理

**导入库**

``` r
library(tidyverse)  # 加载tidyverse包，提供ggplot()、%>%、mutate()等函数，便于数据处理和可视化
library(broom)  # 加载broom包，将模型结果转换为数据框，方便后续分析
library(scales)  # 加载scales包，提供格式化数字的函数，如comma()、percent()和dollar()，用于美化输出
library(modelsummary)  # 加载modelsummary包，创建并排的回归表，便于比较不同模型的结果
```

    Warning message in system("timedatectl", intern = TRUE):
    “running command 'timedatectl' had status 1”
    ── [1mAttaching packages[22m ─────────────────────────────────────── tidyverse 1.3.1 ──

    [32m✔[39m [34mggplot2[39m 3.4.4     [32m✔[39m [34mpurrr  [39m 1.0.2
    [32m✔[39m [34mtibble [39m 3.2.1     [32m✔[39m [34mdplyr  [39m 1.1.4
    [32m✔[39m [34mtidyr  [39m 1.3.0     [32m✔[39m [34mstringr[39m 1.5.1
    [32m✔[39m [34mreadr  [39m 2.1.2     [32m✔[39m [34mforcats[39m 0.5.1

    ── [1mConflicts[22m ────────────────────────────────────────── tidyverse_conflicts() ──
    [31m✖[39m [34mdplyr[39m::[32mfilter()[39m masks [34mstats[39m::filter()
    [31m✖[39m [34mdplyr[39m::[32mlag()[39m    masks [34mstats[39m::lag()

​  
​Attaching package: ‘scales’

​  
​The following object is masked from ‘package:purrr’: ​  
discard

​  
​The following object is masked from ‘package:readr’: ​  
col_factor

​

**导入数据**

``` r
# 加载数据
# 使用read_csv函数从指定路径读取injury.csv文件，并将其存储在injury_raw变量中
injury_raw <- read_csv("data/injury.csv")
```

    [1mRows: [22m[34m7150[39m [1mColumns: [22m[34m30[39m
    [36m──[39m [1mColumn specification[22m [36m────────────────────────────────────────────────────────[39m
    [1mDelimiter:[22m ","
    [32mdbl[39m (30): durat, afchnge, highearn, male, married, hosp, indust, injtype, ag...

    [36mℹ[39m Use `spec()` to retrieve the full column specification for this data.
    [36mℹ[39m Specify the column types or set `show_col_types = FALSE` to quiet this message.

``` r
# 清理数据
# 过滤出仅来自肯塔基州的观察结果，ky == 1表示只选择肯塔基州的数据
injury <- injury_raw %>% 
  filter(ky == 1) %>% 
  # 重命名变量以便于理解
  # 语法为 `新名称 = 原名称`
  rename(duration = durat,  # 将durat重命名为duration，表示持续时间
         log_duration = ldurat,  # 将ldurat重命名为log_duration，表示持续时间的对数
         after_1980 = afchnge)  # 将afchnge重命名为after_1980，表示是否在1980年后发生变化
```

### 探索性数据分析

首先，我们可以看看高收入者和低收入者（我们的对照组和处理组）之间失业救济金的分布情况。

``` r
# 可视化高收入者和低收入者之间失业救济金的分布情况
ggplot(data = injury, aes(x = duration)) +  # 使用ggplot绘制图形，x轴为duration
  # 设置直方图的参数
  # binwidth = 8表示每个柱子代表2个月（8周）
  # boundary = 0确保0-8的柱子从0开始，而不是-4到4
  geom_histogram(binwidth = 8, color = "white", boundary = 0) +  # 绘制直方图
  # 按照高收入者和低收入者分面显示
  facet_wrap(vars(highearn))  # vars(highearn)用于根据highearn变量分面
```

<figure>
<img
src="http://www.plutoese.com:8888/images/2025/05/20/output_10_0.png"
alt="png" />
<figcaption aria-hidden="true">png</figcaption>
</figure>

如果我们使用持续时间的对数，我们可以得到一个偏斜度较小的分布，它与回归模型配合得更好。

``` r
# 如果我们使用持续时间的对数，我们可以得到一个偏斜度较小的分布，它与回归模型配合得更好。
ggplot(data = injury, mapping = aes(x = log_duration)) +  # 使用log_duration作为x轴
  geom_histogram(binwidth = 0.5, color = "white", boundary = 0) +  # 绘制直方图
  # 如果希望在x轴上对数值进行指数化，可以取消注释以下行
  # 这将使得x轴显示e^1, e^2, e^3等，而不是1, 2, 3等，使标签更易读
  # scale_x_continuous(labels = trans_format("exp", format = round)) +
  facet_wrap(vars(highearn))  # 根据highearn变量分面显示
```

<figure>
<img
src="http://www.plutoese.com:8888/images/2025/05/20/output_12_0.png"
alt="png" />
<figcaption aria-hidden="true">png</figcaption>
</figure>

我们还应该检查政策变化前后的失业分布情况。

``` r
plot_data <- injury %>% 
  # 将highearn和after_1980转换为因子，以便在图中显示更美观的标签
  mutate(highearn = factor(highearn, labels = c("Low earner", "High earner")),  # 将highearn转换为因子并重命名
         after_1980 = factor(after_1980, labels = c("Before 1980", "After 1980"))) %>%  # 将after_1980转换为因子并重命名
  group_by(highearn, after_1980) %>%  # 按照highearn和after_1980分组
  summarize(mean_duration = mean(log_duration),  # 计算每组的平均对数持续时间
            se_duration = sd(log_duration) / sqrt(n()),  # 计算每组的标准误
            upper = mean_duration + (1.96 * se_duration),  # 计算置信区间上限
            lower = mean_duration - (1.96 * se_duration))  # 计算置信区间下限

# 绘制高收入者和低收入者的平均对数持续时间及其置信区间
ggplot(plot_data, aes(x = highearn, y = mean_duration)) +
  geom_pointrange(aes(ymin = lower, ymax = upper),  # 绘制点范围图，显示置信区间
                  color = "darkgreen", size = 1) +  # 设置颜色和大小
  facet_wrap(vars(after_1980))  # 根据after_1980变量分面显示
```

    [1m[22m`summarise()` has grouped output by 'highearn'. You can override using the
    `.groups` argument.

<figure>
<img
src="http://www.plutoese.com:8888/images/2025/05/20/output_14_1e336c2af8de8b25e.png"
alt="png" />
<figcaption aria-hidden="true">png</figcaption>
</figure>

``` r
# 或者，可以绘制政策变化前后不同收入者的对比
ggplot(plot_data, aes(x = after_1980, y = mean_duration, color = highearn)) +
  geom_pointrange(aes(ymin = lower, ymax = upper), size = 1) +  # 绘制点范围图
  # group = highearn使得线条在不同类别之间连接
  geom_line(aes(group = highearn))  # 绘制连接线
```

<figure>
<img
src="http://www.plutoese.com:8888/images/2025/05/20/output_15_0.png"
alt="png" />
<figcaption aria-hidden="true">png</figcaption>
</figure>

### 计算DID

$$
\begin{aligned}
\log (\text { duration })= & \alpha+\beta \text { highearn }+\gamma \text { after_1980+ } \\
& \delta\left(\text { highearn } \times \text { after_1980 } \right)+\epsilon
\end{aligned}
$$

``` r
# 计算DID估计量
model_small <- lm(log_duration ~ highearn + after_1980 + highearn * after_1980,
                  data = injury)  # 使用线性模型计算log_duration与highearn、after_1980及其交互项的关系
tidy(model_small)  # 将模型结果整理为数据框格式
```

<table class="dataframe">
<caption>
A tibble: 4 × 5
</caption>
<thead>
<tr>
<th scope="col">
term
</th>
<th scope="col">
estimate
</th>
<th scope="col">
std.error
</th>
<th scope="col">
statistic
</th>
<th scope="col">
p.value
</th>
</tr>
<tr>
<th scope="col">
\<chr\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
(Intercept)
</td>
<td>
1.125615409
</td>
<td>
0.03073683
</td>
<td>
36.6210680
</td>
<td>
1.617404e-263
</td>
</tr>
<tr>
<td>
highearn
</td>
<td>
0.256478532
</td>
<td>
0.04744641
</td>
<td>
5.4056464
</td>
<td>
6.723705e-08
</td>
</tr>
<tr>
<td>
after_1980
</td>
<td>
0.007657313
</td>
<td>
0.04471726
</td>
<td>
0.1712384
</td>
<td>
8.640425e-01
</td>
</tr>
<tr>
<td>
highearn:after_1980
</td>
<td>
0.190601201
</td>
<td>
0.06850891
</td>
<td>
2.7821376
</td>
<td>
5.418222e-03
</td>
</tr>
</tbody>
</table>

``` r
# 加上控制变量
injury_fixed <- injury %>% 
  mutate(indust = as.factor(indust),  # 将indust转换为因子，以便于模型分析
         injtype = as.factor(injtype))  # 将injtype转换为因子，以便于模型分析
model_big <- lm(log_duration ~ highearn + after_1980 + highearn * after_1980 + 
                  male + married + age + hosp + indust + injtype + lprewage,
                data = injury_fixed)  # 使用线性模型计算包含控制变量的log_duration
tidy(model_big)  # 将模型结果整理为数据框格式
```

<table class="dataframe">
<caption>
A tibble: 18 × 5
</caption>
<thead>
<tr>
<th scope="col">
term
</th>
<th scope="col">
estimate
</th>
<th scope="col">
std.error
</th>
<th scope="col">
statistic
</th>
<th scope="col">
p.value
</th>
</tr>
<tr>
<th scope="col">
\<chr\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
(Intercept)
</td>
<td>
-1.528201819
</td>
<td>
0.422211753
</td>
<td>
-3.619515
</td>
<td>
2.978948e-04
</td>
</tr>
<tr>
<td>
highearn
</td>
<td>
-0.151780763
</td>
<td>
0.089115617
</td>
<td>
-1.703189
</td>
<td>
8.859096e-02
</td>
</tr>
<tr>
<td>
after_1980
</td>
<td>
0.049539510
</td>
<td>
0.041321165
</td>
<td>
1.198889
</td>
<td>
2.306242e-01
</td>
</tr>
<tr>
<td>
male
</td>
<td>
-0.084288778
</td>
<td>
0.042314659
</td>
<td>
-1.991952
</td>
<td>
4.642725e-02
</td>
</tr>
<tr>
<td>
married
</td>
<td>
0.056662325
</td>
<td>
0.037304656
</td>
<td>
1.518908
</td>
<td>
1.288451e-01
</td>
</tr>
<tr>
<td>
age
</td>
<td>
0.006506964
</td>
<td>
0.001337941
</td>
<td>
4.863415
</td>
<td>
1.186953e-06
</td>
</tr>
<tr>
<td>
hosp
</td>
<td>
1.130493166
</td>
<td>
0.037006277
</td>
<td>
30.548687
</td>
<td>
5.199920e-189
</td>
</tr>
<tr>
<td>
indust2
</td>
<td>
0.183864193
</td>
<td>
0.054130748
</td>
<td>
3.396668
</td>
<td>
6.871068e-04
</td>
</tr>
<tr>
<td>
indust3
</td>
<td>
0.163485342
</td>
<td>
0.037852082
</td>
<td>
4.319058
</td>
<td>
1.595428e-05
</td>
</tr>
<tr>
<td>
injtype2
</td>
<td>
0.935467528
</td>
<td>
0.143730662
</td>
<td>
6.508476
</td>
<td>
8.287217e-11
</td>
</tr>
<tr>
<td>
injtype3
</td>
<td>
0.635465944
</td>
<td>
0.085441798
</td>
<td>
7.437413
</td>
<td>
1.190161e-13
</td>
</tr>
<tr>
<td>
injtype4
</td>
<td>
0.554549906
</td>
<td>
0.092849322
</td>
<td>
5.972579
</td>
<td>
2.486502e-09
</td>
</tr>
<tr>
<td>
injtype5
</td>
<td>
0.641201298
</td>
<td>
0.085434855
</td>
<td>
7.505149
</td>
<td>
7.150444e-14
</td>
</tr>
<tr>
<td>
injtype6
</td>
<td>
0.615041146
</td>
<td>
0.086312085
</td>
<td>
7.125782
</td>
<td>
1.172582e-12
</td>
</tr>
<tr>
<td>
injtype7
</td>
<td>
0.991335785
</td>
<td>
0.190531564
</td>
<td>
5.203000
</td>
<td>
2.034395e-07
</td>
</tr>
<tr>
<td>
injtype8
</td>
<td>
0.434082063
</td>
<td>
0.118911861
</td>
<td>
3.650452
</td>
<td>
2.642875e-04
</td>
</tr>
<tr>
<td>
lprewage
</td>
<td>
0.284480637
</td>
<td>
0.080055679
</td>
<td>
3.553535
</td>
<td>
3.833831e-04
</td>
</tr>
<tr>
<td>
highearn:after_1980
</td>
<td>
0.168721341
</td>
<td>
0.063974963
</td>
<td>
2.637303
</td>
<td>
8.381088e-03
</td>
</tr>
</tbody>
</table>

``` r
# 使用modelsummary包创建并排的回归表，便于比较不同模型的结果
modelsummary(list("Simple" = model_small, "Full" = model_big), output = "markdown")
```

​  
​  
​\| \| Simple \| Full \| ​ \|:———————\|:——-:\|:——-:\| ​ \|(Intercept) \|
1.126 \| -1.528 \| ​ \| \| (0.031) \| (0.422) \| ​ \|highearn \| 0.256 \|
-0.152 \| ​ \| \| (0.047) \| (0.089) \| ​ \|after_1980 \| 0.008 \| 0.050
\| ​ \| \| (0.045) \| (0.041) \| ​ \|highearn × after_1980 \| 0.191 \|
0.169 \| ​ \| \| (0.069) \| (0.064) \| ​ \|male \| \| -0.084 \| ​ \| \| \|
(0.042) \| ​ \|married \| \| 0.057 \| ​ \| \| \| (0.037) \| ​ \|age \| \|
0.007 \| ​ \| \| \| (0.001) \| ​ \|hosp \| \| 1.130 \| ​ \| \| \| (0.037)
\| ​ \|indust2 \| \| 0.184 \| ​ \| \| \| (0.054) \| ​ \|indust3 \| \| 0.163
\| ​ \| \| \| (0.038) \| ​ \|injtype2 \| \| 0.935 \| ​ \| \| \| (0.144) \| ​
\|injtype3 \| \| 0.635 \| ​ \| \| \| (0.085) \| ​ \|injtype4 \| \| 0.555
\| ​ \| \| \| (0.093) \| ​ \|injtype5 \| \| 0.641 \| ​ \| \| \| (0.085) \| ​
\|injtype6 \| \| 0.615 \| ​ \| \| \| (0.086) \| ​ \|injtype7 \| \| 0.991
\| ​ \| \| \| (0.191) \| ​ \|injtype8 \| \| 0.434 \| ​ \| \| \| (0.119) \| ​
\|lprewage \| \| 0.284 \| ​ \| \| \| (0.080) \| ​ \|Num.Obs. \| 5626 \|
5347 \| ​ \|R2 \| 0.021 \| 0.190 \| ​ \|R2 Adj. \| 0.020 \| 0.187 \| ​
\|AIC \| 18654.0 \| 16684.8 \| ​ \|BIC \| 18687.2 \| 16809.9 \| ​ \|F \|
39.540 \| 73.459 \| ​ \|RMSE \| 1.27 \| 1.15 \|

## 多个时期的DID案例

### 预处理

**导入库**

``` r
library(tidyverse)     # 导入tidyverse包，包含ggplot2、dplyr等常用数据处理和可视化工具
library(haven)         # 导入haven包，用于读取Stata文件
library(fixest)        # 导入fixest包，提供固定效应回归的实现
library(estimatr)      # 导入estimatr包，另一种实现固定效应回归的方法
library(broom)         # 导入broom包，将模型对象转换为数据框
library(kableExtra)    # 导入kableExtra包，用于生成美观的表格
library(modelsummary)  # 导入modelsummary包，生成美观的回归结果表
library(patchwork)     # 导入patchwork包，用于组合多个图形
library(scales)        # 导入scales包，提供美观的格式化函数
```

    Attaching package: ‘fixest’

​  
​The following object is masked from ‘package:scales’: ​  
pvalue

​  
​  
​Attaching package: ‘kableExtra’

​  
​The following object is masked from ‘package:dplyr’: ​  
group_rows

​

``` r
# 自定义ggplot主题以生成美观的图形
theme_clean <- function() {
  theme_minimal(base_family = "Barlow Semi Condensed") +  # 使用最小主题，设置基础字体
    theme(panel.grid.minor = element_blank(),  # 隐藏次要网格线
          plot.title = element_text(face = "bold"),  # 设置图表标题为粗体
          axis.title = element_text(family = "Barlow Semi Condensed Medium"),  # 设置坐标轴标题字体
          strip.text = element_text(family = "Barlow Semi Condensed",  # 设置分面标签字体
                                    face = "bold", size = rel(1), hjust = 0),
          strip.background = element_rect(fill = "grey80", color = NA),  # 设置分面背景颜色
          plot.caption = element_text(hjust = 0))  # 设置图表说明的对齐方式
}
```

**生成模拟数据**

``` r
set.seed(1234)  # 设置随机种子以确保结果可重复

n_rows <- 500  # 定义生成数据的行数
fake_data <- tibble(treatment = rbeta(n_rows, shape1 = 3, shape2 = 7)) %>%  # 生成treatment变量，服从Beta分布
  mutate(treatment = round(treatment * 100)) %>%  # 将treatment值缩放并四舍五入到整数
  # 构建结果变量，基于某个基线水平的结果 + 由于处理而增加的结果 + 一些噪声
  mutate(outcome_baseline = rnorm(n_rows, mean = 800, sd = 200),  # 生成基线结果，服从正态分布
         treatment_boost = 10 * treatment,  # 计算处理提升，处理值乘以10
         outcome = outcome_baseline + treatment_boost + rnorm(n_rows, 200, 100)) %>%  # 计算最终结果
  select(outcome, treatment)  # 选择最终结果和处理变量

head(fake_data)  # 显示生成数据的前几行
```

<table class="dataframe">
<caption>
A tibble: 6 × 2
</caption>
<thead>
<tr>
<th scope="col">
outcome
</th>
<th scope="col">
treatment
</th>
</tr>
<tr>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
514.2789
</td>
<td>
13
</td>
</tr>
<tr>
<td>
1550.2972
</td>
<td>
35
</td>
</tr>
<tr>
<td>
1561.7238
</td>
<td>
52
</td>
</tr>
<tr>
<td>
1036.6449
</td>
<td>
4
</td>
</tr>
<tr>
<td>
1136.5141
</td>
<td>
38
</td>
</tr>
<tr>
<td>
1277.1465
</td>
<td>
39
</td>
</tr>
</tbody>
</table>

``` r
# 创建结果变量的直方图
hist_out <- ggplot(fake_data, aes(x = outcome)) +
  geom_histogram(binwidth = 100, color = "white",  # 设置直方图的边框颜色为白色
                 boundary = 0, fill = "#FFA615") +  # 设置直方图的填充颜色
  scale_x_continuous(labels = dollar_format()) +  # 将x轴标签格式化为货币格式
  labs(title = "Distribution of outcome",  # 设置图表标题
       x = "Outcome", y = "Count") +  # 设置x轴和y轴的标签
  coord_cartesian(ylim = c(0, 80)) +  # 设置y轴的显示范围
  theme_clean() +  # 应用自定义主题
  theme(panel.grid.major.x = element_blank())  # 隐藏主要x轴网格线

# 创建处理变量的直方图
hist_trt <- ggplot(fake_data, aes(x = treatment)) +
  geom_histogram(binwidth = 5, color = "white",  # 设置直方图的边框颜色为白色
                 boundary = 0, fill = "#A4BD0A") +  # 设置直方图的填充颜色
  labs(title = "Distribution of treatment",  # 设置图表标题
       x = "Treatment", y = "Count") +  # 设置x轴和y轴的标签
  coord_cartesian(ylim = c(0, 80)) +  # 设置y轴的显示范围
  theme_clean() +  # 应用自定义主题
  theme(panel.grid.major.x = element_blank())  # 隐藏主要x轴网格线

# 创建处理与结果之间关系的散点图
plot_trt_out <- ggplot(fake_data, aes(x = treatment, y = outcome)) +
  geom_point(size = 0.75) +  # 绘制散点，设置点的大小
  geom_smooth(method = "lm", color = "#0599B0") +  # 添加线性回归平滑线
  scale_y_continuous(labels = dollar_format()) +  # 将y轴标签格式化为货币格式
  labs(title = "Effect of treatment on outcome",  # 设置图表标题
       x = "Treatment", y = "Outcome") +  # 设置x轴和y轴的标签
  theme_clean()  # 应用自定义主题

# 组合直方图和散点图，并设置各部分的高度比例
(hist_out + hist_trt) / plot_spacer() / plot_trt_out +
  plot_layout(heights = c(0.28, 0.02, 0.7))  # 设置组合图的高度比例
```

    [1m[22m`geom_smooth()` using formula = 'y ~ x'

<figure>
<img
src="http://www.plutoese.com:8888/images/2025/05/20/output_28_1.png"
alt="png" />
<figcaption aria-hidden="true">png</figcaption>
</figure>

### 关于OLS回归得到的估计量

计算平均处理效应（ATE）

$$
\overbrace{\underbrace{\text { Outcome }}_{\text {An individual }}}^{\text {Intercept }}=\overbrace{\alpha}^\beta+\underbrace{\beta}_{\text {ATE }} \overbrace{D_i}^{\text {Treatment }}+\overbrace{\epsilon_i}^{\text {Error }}
$$

### 进行线性回归分析，模型为结果变量与处理变量的关系

model_effect \<- lm(outcome ~ treatment, data = fake_data)
tidy(model_effect) \# 将模型结果整理为数据框格式

雅基耶拉（2021，4）展示了一种基于弗里希-沃-洛弗尔定理计算$\beta$的替代方法。

$$
\beta=\sum_i Y_i\left(\frac{\tilde{D}_i}{\sum_i \tilde{D}_i^2}\right)
$$

``` r
# 使用另一种方法计算偏回归系数
trt_resid <- lm(treatment ~ 1, data = fake_data)  # 计算处理变量的残差

fake_data_resid <- fake_data %>%
  mutate(treatment_resid = residuals(trt_resid)) %>%  # 添加处理变量的残差
  mutate(mean_treat = mean(treatment),  # 计算处理变量的均值
         diff = treatment - mean_treat)  # 计算处理变量与均值的差异

head(fake_data_resid)  # 显示处理残差数据的前几行
```

<table class="dataframe">
<caption>
A tibble: 6 × 5
</caption>
<thead>
<tr>
<th scope="col">
outcome
</th>
<th scope="col">
treatment
</th>
<th scope="col">
treatment_resid
</th>
<th scope="col">
mean_treat
</th>
<th scope="col">
diff
</th>
</tr>
<tr>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
514.2789
</td>
<td>
13
</td>
<td>
-17.176
</td>
<td>
30.176
</td>
<td>
-17.176
</td>
</tr>
<tr>
<td>
1550.2972
</td>
<td>
35
</td>
<td>
4.824
</td>
<td>
30.176
</td>
<td>
4.824
</td>
</tr>
<tr>
<td>
1561.7238
</td>
<td>
52
</td>
<td>
21.824
</td>
<td>
30.176
</td>
<td>
21.824
</td>
</tr>
<tr>
<td>
1036.6449
</td>
<td>
4
</td>
<td>
-26.176
</td>
<td>
30.176
</td>
<td>
-26.176
</td>
</tr>
<tr>
<td>
1136.5141
</td>
<td>
38
</td>
<td>
7.824
</td>
<td>
30.176
</td>
<td>
7.824
</td>
</tr>
<tr>
<td>
1277.1465
</td>
<td>
39
</td>
<td>
8.824
</td>
<td>
30.176
</td>
<td>
8.824
</td>
</tr>
</tbody>
</table>

``` r
#计算加权回归系数
fake_data_resid %>%
  summarize(beta = sum(outcome * (treatment_resid / sum(treatment_resid^2))))  # 计算加权回归系数
```

<table class="dataframe">
<caption>
A tibble: 1 × 1
</caption>
<thead>
<tr>
<th scope="col">
beta
</th>
</tr>
<tr>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
9.690602
</td>
</tr>
</tbody>
</table>

雅基耶拉以这种方式重新参数化$\beta$的原因是，像这样查看残差会为每个观测值创建固有权重。从本质上讲，$\beta$是结果变量的加权和，权重通过

$$
\frac{\tilde{D}_{i t}}{\sum_{i t} \tilde{D}_{i t}^2}
$$

计算得出，即(treatment_resid /
sum(treatment_resid^2))。我们可以为每个观测值计算权重。结果变量的加权和。

``` r
# 计算处理变量的加权值
fake_data_with_weights <- fake_data_resid %>%
  mutate(treatment_weight = treatment_resid / sum(treatment_resid^2))  # 计算处理变量的加权值
head(fake_data_with_weights)  # 显示加权数据的前几行
```

<table class="dataframe">
<caption>
A tibble: 6 × 6
</caption>
<thead>
<tr>
<th scope="col">
outcome
</th>
<th scope="col">
treatment
</th>
<th scope="col">
treatment_resid
</th>
<th scope="col">
mean_treat
</th>
<th scope="col">
diff
</th>
<th scope="col">
treatment_weight
</th>
</tr>
<tr>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
514.2789
</td>
<td>
13
</td>
<td>
-17.176
</td>
<td>
30.176
</td>
<td>
-17.176
</td>
<td>
-1.676255e-04
</td>
</tr>
<tr>
<td>
1550.2972
</td>
<td>
35
</td>
<td>
4.824
</td>
<td>
30.176
</td>
<td>
4.824
</td>
<td>
4.707880e-05
</td>
</tr>
<tr>
<td>
1561.7238
</td>
<td>
52
</td>
<td>
21.824
</td>
<td>
30.176
</td>
<td>
21.824
</td>
<td>
2.129867e-04
</td>
</tr>
<tr>
<td>
1036.6449
</td>
<td>
4
</td>
<td>
-26.176
</td>
<td>
30.176
</td>
<td>
-26.176
</td>
<td>
-2.554591e-04
</td>
</tr>
<tr>
<td>
1136.5141
</td>
<td>
38
</td>
<td>
7.824
</td>
<td>
30.176
</td>
<td>
7.824
</td>
<td>
7.635665e-05
</td>
</tr>
<tr>
<td>
1277.1465
</td>
<td>
39
</td>
<td>
8.824
</td>
<td>
30.176
</td>
<td>
8.824
</td>
<td>
8.611594e-05
</td>
</tr>
</tbody>
</table>

我们可以可视化权重的分布。

### 创建处理加权值的直方图

ggplot(fake_data_with_weights, aes(x = treatment_weight)) +
geom_histogram(binwidth = 0.00005, color = “white”, \#
设置直方图的边框颜色为白色 boundary = 0, fill = “#F6C848”) + \#
设置直方图的填充颜色 geom_vline(xintercept = 0, color = “#FF2E00”,
linewidth = 1) + \# 添加垂直线表示零值 labs(x = “Residualized treatment
weight”, y = “Count”) + \# 设置x轴和y轴的标签 scale_x_continuous(labels
= comma_format()) + \# 格式化x轴标签 theme_clean() + \# 应用自定义主题
theme(panel.grid.major.x = element_blank()) \# 隐藏主要x轴网格线

查看上面fake_data_with_weights的前几行
——treatment_resid为负的观测值具有负权重。

### 双重差分模型的残差化权重

这种根据处理状态对结果变量进行加权的想法（处理单元获得正权重，未处理单元获得负权重）是雅基耶拉其余论点和诊断的核心（以及所有其他出色的新双重差分论文）。从根本上说，双重差分估计在不同时间进行处理时变得奇怪和有偏差的主要原因是权重问题。

#### 新案例

**预处理**

``` r
# 读取WDI-FPE数据集
fpe_raw <- read_dta("data/WDI-FPE-data.dta")

# 过滤掉小学入学率缺失的行
fpe_primary <- fpe_raw %>%
  filter(!is.na(primary))  # 仅保留小学入学率不为NA的行

# 过滤掉中学入学率缺失的行
fpe_secondary <- fpe_raw %>%
  filter(!is.na(secondary))  # 仅保留中学入学率不为NA的行

# 显示处理后小学入学率数据的前几行
head(fpe_primary)
```

<table class="dataframe">
<caption>
A tibble: 6 × 8
</caption>
<thead>
<tr>
<th scope="col">
year
</th>
<th scope="col">
country
</th>
<th scope="col">
ccode
</th>
<th scope="col">
primary
</th>
<th scope="col">
id
</th>
<th scope="col">
secondary
</th>
<th scope="col">
fpe_year
</th>
<th scope="col">
treatment
</th>
</tr>
<tr>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<chr\>
</th>
<th scope="col">
\<chr\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
1981
</td>
<td>
Benin
</td>
<td>
BEN
</td>
<td>
60.34757
</td>
<td>
2
</td>
<td>
15.45169
</td>
<td>
2006
</td>
<td>
0
</td>
</tr>
<tr>
<td>
1982
</td>
<td>
Benin
</td>
<td>
BEN
</td>
<td>
64.67790
</td>
<td>
2
</td>
<td>
18.48749
</td>
<td>
2006
</td>
<td>
0
</td>
</tr>
<tr>
<td>
1983
</td>
<td>
Benin
</td>
<td>
BEN
</td>
<td>
66.25000
</td>
<td>
2
</td>
<td>
21.22417
</td>
<td>
2006
</td>
<td>
0
</td>
</tr>
<tr>
<td>
1984
</td>
<td>
Benin
</td>
<td>
BEN
</td>
<td>
64.19849
</td>
<td>
2
</td>
<td>
20.14872
</td>
<td>
2006
</td>
<td>
0
</td>
</tr>
<tr>
<td>
1985
</td>
<td>
Benin
</td>
<td>
BEN
</td>
<td>
64.30680
</td>
<td>
2
</td>
<td>
18.99198
</td>
<td>
2006
</td>
<td>
0
</td>
</tr>
<tr>
<td>
1986
</td>
<td>
Benin
</td>
<td>
BEN
</td>
<td>
62.36633
</td>
<td>
2
</td>
<td>
16.34088
</td>
<td>
2006
</td>
<td>
0
</td>
</tr>
</tbody>
</table>

| 变量名               | 解释                                           |
|----------------------|------------------------------------------------|
| year                 | 观测的年份                                     |
| country & ccode & id | 国家名称、ISO3国家代码以及每个国家的编号       |
| primary              | 小学的总入学率                                 |
| secondary            | 中学的总入学率                                 |
| fpe_year             | 国家取消小学费用的年份                         |
| treatment            | 指示变量，当`year`大于`fpe_year`时为1，否则为0 |

模型可以写为

$$
\overbrace{\underbrace{\text { Outcome }}_{\substack{i t}} \begin{array}{c}
\text { country } \\
t: \text { year }
\end{array}}^{\text {Intercept }}=\overbrace{\alpha}^{\text {In }}+\overbrace{\lambda_i}^{\begin{array}{c}
\text { Country } \\
\text { fixed } \\
\text { effects }
\end{array}}+\overbrace{\gamma_t}^{\text {Year }}+\underbrace{\beta}_{\text {ATE }} \text { fixed } \quad \overbrace{D_{i t}}^{\text {effects }}+\overbrace{\boldsymbol{\epsilon}_{i t}}^{\text {Treatment }}
$$

或者简写为

$$
Y_{i t}=\alpha+\lambda_i+\gamma_t+\beta D_{i t}+\epsilon_{i t}
$$

使用`lm()`进行线性回归

``` r
# 使用线性模型分析小学入学率与处理变量、国家和年份的关系
model_lm <- lm(primary ~ treatment + country + factor(year),
               data = fpe_primary)

# 将模型结果整理为数据框格式，并过滤掉国家和年份的项
tidy(model_lm) %>%
  filter(!str_detect(term, "country"), !str_detect(term, "year"))

# 获取模型的摘要信息，包括R方等统计量
glance(model_lm)
```

<table class="dataframe">
<caption>
A tibble: 2 × 5
</caption>
<thead>
<tr>
<th scope="col">
term
</th>
<th scope="col">
estimate
</th>
<th scope="col">
std.error
</th>
<th scope="col">
statistic
</th>
<th scope="col">
p.value
</th>
</tr>
<tr>
<th scope="col">
\<chr\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
(Intercept)
</td>
<td>
72.38045
</td>
<td>
4.632129
</td>
<td>
15.625740
</td>
<td>
4.229903e-44
</td>
</tr>
<tr>
<td>
treatment
</td>
<td>
20.42817
</td>
<td>
2.750611
</td>
<td>
7.426773
</td>
<td>
5.823416e-13
</td>
</tr>
</tbody>
</table>
<table class="dataframe">
<caption>
A tibble: 1 × 12
</caption>
<thead>
<tr>
<th scope="col">
r.squared
</th>
<th scope="col">
adj.r.squared
</th>
<th scope="col">
sigma
</th>
<th scope="col">
statistic
</th>
<th scope="col">
p.value
</th>
<th scope="col">
df
</th>
<th scope="col">
logLik
</th>
<th scope="col">
AIC
</th>
<th scope="col">
BIC
</th>
<th scope="col">
deviance
</th>
<th scope="col">
df.residual
</th>
<th scope="col">
nobs
</th>
</tr>
<tr>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<int\>
</th>
<th scope="col">
\<int\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
0.7682702
</td>
<td>
0.7424639
</td>
<td>
14.68051
</td>
<td>
29.77067
</td>
<td>
1.303227e-110
</td>
<td>
49
</td>
<td>
-1985.306
</td>
<td>
4072.611
</td>
<td>
4286.526
</td>
<td>
94827.66
</td>
<td>
440
</td>
<td>
490
</td>
</tr>
</tbody>
</table>

考虑聚类稳健标准误

``` r
# 计算不同国家的数量，以便后续聚类调整
df_primary <- fpe_primary %>% distinct(country) %>% nrow()

# 进行聚类稳健性检验，使用国家作为聚类变量
model_lm_clustered <- lmtest::coeftest(model_lm,
                                       vcov = sandwich::vcovCL,  # 使用聚类稳健的协方差矩阵
                                       cluster = ~country,  # 指定聚类变量为国家
                                       df = df_primary - 1,  # 自由度调整
                                       # 保留原始模型以便modelsummary显示R2
                                       save = TRUE)

# 将聚类调整后的模型结果整理为数据框格式，并过滤掉国家和年份的项
tidy(model_lm_clustered) %>%
  filter(!str_detect(term, "country"), !str_detect(term, "year"))
```

<table class="dataframe">
<caption>
A tibble: 2 × 5
</caption>
<thead>
<tr>
<th scope="col">
term
</th>
<th scope="col">
estimate
</th>
<th scope="col">
std.error
</th>
<th scope="col">
statistic
</th>
<th scope="col">
p.value
</th>
</tr>
<tr>
<th scope="col">
\<chr\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
(Intercept)
</td>
<td>
72.38045
</td>
<td>
5.828196
</td>
<td>
12.419014
</td>
<td>
6.013092e-09
</td>
</tr>
<tr>
<td>
treatment
</td>
<td>
20.42817
</td>
<td>
9.120319
</td>
<td>
2.239852
</td>
<td>
4.184655e-02
</td>
</tr>
</tbody>
</table>

也可以使用更高级的回归函数，这些函数可以更系统地处理固定效应。来自
“estimatr”
包的lm_robust()函数在定义特殊固定效应方面非常有效，这些固定效应会自动从结果表中省略，并且还会返回稳健的、可选择聚类的标准误差。

``` r
# 使用稳健线性模型分析小学入学率与处理变量的关系
# lm_robust函数用于拟合线性模型，考虑固定效应和聚类
model_lm_robust <- lm_robust(primary ~ treatment,  # 设定因变量为小学入学率，自变量为处理变量
                             fixed_effects = ~ country + year,  # 指定国家和年份作为固定效应
                             data = fpe_primary,  # 使用处理后的小学入学率数据
                             clusters = country, se_type = "stata")  # 指定聚类变量为国家，并使用Stata风格的标准误

# 将模型结果整理为数据框格式，便于查看和分析
tidy(model_lm_robust)

# 获取模型的摘要信息，包括R方等统计量
glance(model_lm_robust)
```

<table class="dataframe">
<caption>
A data.frame: 1 × 9
</caption>
<thead>
<tr>
<th scope="col">
term
</th>
<th scope="col">
estimate
</th>
<th scope="col">
std.error
</th>
<th scope="col">
statistic
</th>
<th scope="col">
p.value
</th>
<th scope="col">
conf.low
</th>
<th scope="col">
conf.high
</th>
<th scope="col">
df
</th>
<th scope="col">
outcome
</th>
</tr>
<tr>
<th scope="col">
\<chr\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<chr\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
treatment
</td>
<td>
20.42817
</td>
<td>
9.120319
</td>
<td>
2.239852
</td>
<td>
0.04184655
</td>
<td>
0.8670274
</td>
<td>
39.9893
</td>
<td>
14
</td>
<td>
primary
</td>
</tr>
</tbody>
</table>
<table class="dataframe">
<caption>
A data.frame: 1 × 7
</caption>
<thead>
<tr>
<th scope="col">
r.squared
</th>
<th scope="col">
adj.r.squared
</th>
<th scope="col">
statistic
</th>
<th scope="col">
p.value
</th>
<th scope="col">
df.residual
</th>
<th scope="col">
nobs
</th>
<th scope="col">
se_type
</th>
</tr>
<tr>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<int\>
</th>
<th scope="col">
\<chr\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
0.7682702
</td>
<td>
0.7424639
</td>
<td>
NA
</td>
<td>
NA
</td>
<td>
14
</td>
<td>
490
</td>
<td>
stata
</td>
</tr>
</tbody>
</table>

来自 fixest 包的 feols ()
函数也表现良好，不过在像我们这里有具有嵌套固定效应的小样本时，你需要更改其中一个默认设置才能获得与
Stata 相同的标准误差。

``` r
# 使用feols函数拟合固定效应模型，分析小学入学率与处理变量的关系
# 公式为：因变量为小学入学率(primary)，自变量为处理变量(treatment)
# 固定效应包括国家(country)和年份(year)，以控制这些变量的影响
model_feols <- feols(primary ~ treatment | country + year,
                     data = fpe_primary,  # 使用处理后的小学入学率数据
                     cluster = ~ country,  # 指定聚类变量为国家，以获得稳健的标准误
                     ssc = ssc(fixef.K = "full"))  # 自由度调整，使用完整的固定效应

# 将模型结果整理为数据框格式，便于查看和分析
tidy(model_feols)

# 获取模型的摘要信息，包括R方等统计量
glance(model_feols)
```

<table class="dataframe">
<caption>
A tibble: 1 × 5
</caption>
<thead>
<tr>
<th scope="col">
term
</th>
<th scope="col">
estimate
</th>
<th scope="col">
std.error
</th>
<th scope="col">
statistic
</th>
<th scope="col">
p.value
</th>
</tr>
<tr>
<th scope="col">
\<chr\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
treatment
</td>
<td>
20.42817
</td>
<td>
9.120319
</td>
<td>
2.239852
</td>
<td>
0.04184655
</td>
</tr>
</tbody>
</table>
<table class="dataframe">
<caption>
A tibble: 1 × 9
</caption>
<thead>
<tr>
<th scope="col">
r.squared
</th>
<th scope="col">
adj.r.squared
</th>
<th scope="col">
within.r.squared
</th>
<th scope="col">
pseudo.r.squared
</th>
<th scope="col">
sigma
</th>
<th scope="col">
nobs
</th>
<th scope="col">
AIC
</th>
<th scope="col">
BIC
</th>
<th scope="col">
logLik
</th>
</tr>
<tr>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<int\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
0.7682702
</td>
<td>
0.7424639
</td>
<td>
0.1113929
</td>
<td>
NA
</td>
<td>
14.68051
</td>
<td>
490
</td>
<td>
4070.611
</td>
<td>
4280.331
</td>
<td>
-1985.306
</td>
</tr>
</tbody>
</table>

查看回归表格如下

``` r
# 观察中学的情况
model_lm_sec <- lm(secondary ~ treatment + country + factor(year),  # 拟合线性模型，因变量为中学入学率，包含处理变量、国家和年份作为自变量
                   data = fpe_secondary)  # 使用处理后中学入学率的数据

# 计算不同国家的数量，以便后续聚类调整
df_secondary <- fpe_secondary %>% distinct(country) %>% nrow()  # 获取唯一国家的数量

# 进行聚类稳健性检验，使用国家作为聚类变量
model_lm_clustered_sec <- lmtest::coeftest(model_lm_sec,  # 对拟合的线性模型进行稳健性检验
                                           vcov = sandwich::vcovCL,  # 使用聚类稳健的协方差矩阵
                                           cluster = ~country,  # 指定聚类变量为国家
                                           df = df_secondary - 1,  # 自由度调整
                                           save = TRUE)  # 保存原始模型以便后续使用

# 使用稳健线性模型分析中学入学率与处理变量的关系
model_lm_robust_sec <- lm_robust(secondary ~ treatment,  # 设定因变量为中学入学率，自变量为处理变量
                                 fixed_effects = ~ country + year,  # 指定国家和年份作为固定效应
                                 data = fpe_secondary,  # 使用处理后中学入学率的数据
                                 clusters = country, se_type = "stata")  # 指定聚类变量为国家，并使用Stata风格的标准误

# 使用feols函数拟合固定效应模型，分析中学入学率与处理变量的关系
model_feols_sec <- feols(secondary ~ treatment | country + year,  # 公式为：因变量为中学入学率，自变量为处理变量，固定效应包括国家和年份
                         data = fpe_secondary,  # 使用处理后中学入学率的数据
                         cluster = ~ country,  # 指定聚类变量为国家，以获得稳健的标准误
                         ssc = ssc(fixef.K = "full"))  # 自由度调整，使用完整的固定效应

# 定义要包含的拟合优度统计量
gof_stuff <- tribble(
  ~raw, ~clean, ~fmt,  # 定义原始名称、清晰名称和格式
  "nobs", "N", 0,  # 观测数量
  "r.squared", "R²", 3  # R方值
)

# 定义表格末尾的额外行
extra_rows <- tribble(
  ~term, ~a, ~b, ~c, ~d, ~e, ~f,  # 定义额外行的列
  "Country fixed effects", "•", "•", "•", "•", "•", "•",  # 国家固定效应
  "Year fixed effects", "•", "•", "•", "•", "•", "•"  # 年份固定效应
)

# 汇总模型结果并生成表格
modelsummary(list("lm" = model_lm_clustered,  # 将不同模型的结果放入列表中
                  "lm_robust" = model_lm_robust,
                  "feols" = model_feols,
                  "lm" = model_lm_clustered_sec,
                  "lm_robust" = model_lm_robust_sec,
                  "feols" = model_feols_sec),
             coef_rename = c("treatment" = "Treatment"),  # 重命名系数
             estimate = "{estimate}",  # 估计值的格式
             statistic = c("s.e. = {std.error}", "p = {p.value}"),  # 统计量的格式
             coef_omit = "^country|^factor|Intercept",  # 省略的系数
             gof_map = gof_stuff,  # 拟合优度统计量
             add_rows = extra_rows,  # 添加额外行
             escape = FALSE, output = "jupyter")  # 输出格式设置为markdown
```

<table class="table" style="width: auto !important; margin-left: auto; margin-right: auto;">
<thead>
<tr>
<th style="text-align:left;">
</th>
<th style="text-align:center;">
lm
</th>
<th style="text-align:center;">
lm_robust
</th>
<th style="text-align:center;">
feols
</th>
<th style="text-align:center;">
lm
</th>
<th style="text-align:center;">
lm_robust
</th>
<th style="text-align:center;">
feols
</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;">
Treatment
</td>
<td style="text-align:center;">
20.428
</td>
<td style="text-align:center;">
20.428
</td>
<td style="text-align:center;">
20.428
</td>
<td style="text-align:center;">
−0.468
</td>
<td style="text-align:center;">
−0.468
</td>
<td style="text-align:center;">
−0.468
</td>
</tr>
<tr>
<td style="text-align:left;">
</td>
<td style="text-align:center;">
s.e. = 9.120
</td>
<td style="text-align:center;">
s.e. = 9.120
</td>
<td style="text-align:center;">
s.e. = 9.120
</td>
<td style="text-align:center;">
s.e. = 3.081
</td>
<td style="text-align:center;">
s.e. = 3.081
</td>
<td style="text-align:center;">
s.e. = 3.081
</td>
</tr>
<tr>
<td style="text-align:left;box-shadow: 0px 1px">
</td>
<td style="text-align:center;box-shadow: 0px 1px">
p = 0.042
</td>
<td style="text-align:center;box-shadow: 0px 1px">
p = 0.026
</td>
<td style="text-align:center;box-shadow: 0px 1px">
p = 0.042
</td>
<td style="text-align:center;box-shadow: 0px 1px">
p = 0.881
</td>
<td style="text-align:center;box-shadow: 0px 1px">
p = 0.879
</td>
<td style="text-align:center;box-shadow: 0px 1px">
p = 0.881
</td>
</tr>
<tr>
<td style="text-align:left;">
N
</td>
<td style="text-align:center;">
490
</td>
<td style="text-align:center;">
490
</td>
<td style="text-align:center;">
490
</td>
<td style="text-align:center;">
369
</td>
<td style="text-align:center;">
369
</td>
<td style="text-align:center;">
369
</td>
</tr>
<tr>
<td style="text-align:left;">
R²
</td>
<td style="text-align:center;">
</td>
<td style="text-align:center;">
0.768
</td>
<td style="text-align:center;">
0.768
</td>
<td style="text-align:center;">
</td>
<td style="text-align:center;">
0.942
</td>
<td style="text-align:center;">
0.942
</td>
</tr>
<tr>
<td style="text-align:left;">
Country fixed effects
</td>
<td style="text-align:center;">
•
</td>
<td style="text-align:center;">
•
</td>
<td style="text-align:center;">
•
</td>
<td style="text-align:center;">
•
</td>
<td style="text-align:center;">
•
</td>
<td style="text-align:center;">
•
</td>
</tr>
<tr>
<td style="text-align:left;">
Year fixed effects
</td>
<td style="text-align:center;">
•
</td>
<td style="text-align:center;">
•
</td>
<td style="text-align:center;">
•
</td>
<td style="text-align:center;">
•
</td>
<td style="text-align:center;">
•
</td>
<td style="text-align:center;">
•
</td>
</tr>
</tbody>
</table>

这些回归使用 lm () 或 lm_robust ()
运行起来足够简单，但这里存在一个时间差异的问题。这 15
个国家各自在不同的年份通过了取消费用的法律

``` r
# 从原始数据中筛选出特定年份的数据
plot_fpe_start <- fpe_raw %>%
  filter(year == fpe_year) %>%  # 过滤出年份等于fpe_year的行
  arrange(fpe_year, country) %>%  # 按照年份和国家进行排序
  mutate(country = fct_inorder(country))  # 将国家因子按出现顺序进行排序

# 使用ggplot2绘制图形
ggplot(plot_fpe_start, aes(y = fct_rev(country))) +  # 设置y轴为反向的国家因子
  geom_segment(aes(x = fpe_year, xend = 2015, yend = country),  # 绘制线段，表示从实施年份到2015年的变化
               size = 3, color = "#D6EB52") +  # 设置线段的粗细和颜色
  geom_text(aes(x = fpe_year + 0.2), label = "▶", family = "Arial Unicode MS",  # 在实施年份旁边添加箭头符号
            size = 8, color = "#A4BD0A") +  # 设置箭头的大小和颜色
  labs(x = "Year free primary education implemented", y = NULL) +  # 设置x轴标签
  theme_clean()  # 应用清晰的主题样式
```

    Warning message:
    “[1m[22mUsing `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
    [36mℹ[39m Please use `linewidth` instead.”

<figure>
<img
src="http://www.plutoese.com:8888/images/2025/05/20/output_58_1.png"
alt="png" />
<figcaption aria-hidden="true">png</figcaption>
</figure>

在双向固定效应（TWFE）情况下，处理残差需要考虑国家和年份的固定效应，通过
lm (treatment ~ country + year) 来计算。这种差异是因为 TWFE
情况中时间差异可能会导致问题，为了查看这些问题，使用带有残差化处理权重的弗里希 -
沃 - 洛夫尔定理（Frisch-Waugh-Lovell theorem）计算
ATE，此时对处理残差的计算方法有所不同。

``` r
# 使用线性模型计算处理变量的残差，考虑国家和年份的固定效应
trt_resid_primary <- lm(treatment ~ country + factor(year), data = fpe_primary)  # 针对初级教育数据
trt_resid_secondary <- lm(treatment ~ country + factor(year), data = fpe_secondary)  # 针对中级教育数据

# 计算初级教育的权重
fpe_primary_weights <- fpe_primary %>%
  mutate(treatment_resid = residuals(trt_resid_primary)) %>%  # 获取处理变量的残差
  mutate(treatment_weight = treatment_resid / sum(treatment_resid^2))  # 计算权重，归一化处理变量的残差

# 计算中级教育的权重
fpe_secondary_weights <- fpe_secondary %>%
  mutate(treatment_resid = residuals(trt_resid_secondary)) %>%  # 获取处理变量的残差
  mutate(treatment_weight = treatment_resid / sum(treatment_resid^2))  # 计算权重，归一化处理变量的残差

# 汇总初级教育的加权结果
fpe_primary_weights %>%
  summarize(twfe_beta_primary = sum(primary * treatment_weight))  # 计算加权后的初级教育效果
```

<table class="dataframe">
<caption>
A tibble: 1 × 1
</caption>
<thead>
<tr>
<th scope="col">
twfe_beta_primary
</th>
</tr>
<tr>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
20.42817
</td>
</tr>
</tbody>
</table>

``` r
# 汇总中级教育的加权结果
fpe_secondary_weights %>%
  summarize(twfe_beta_secondary = sum(secondary * treatment_weight))  # 计算加权后的中级教育效果
```

<table class="dataframe">
<caption>
A tibble: 1 × 1
</caption>
<thead>
<tr>
<th scope="col">
twfe_beta_secondary
</th>
</tr>
<tr>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
-0.4684782
</td>
</tr>
</tbody>
</table>

### 雅基拉诊断法

**处理后的观测值会得到负权重吗？**

``` r
# 计算初级教育数据中接受处理的总人数
n_treated_primary <- fpe_primary_weights %>%
  filter(treatment == 1) %>%  # 过滤出处理变量为1的行
  nrow()  # 计算符合条件的行数

# 计算中级教育数据中接受处理的总人数
n_treated_secondary <- fpe_secondary_weights %>%
  filter(treatment == 1) %>%  # 过滤出处理变量为1的行
  nrow()  # 计算符合条件的行数

# 计算初级教育数据中负权重的处理观察值数量
n_treated_negative_primary <- fpe_primary_weights %>%
  filter(treatment_weight < 0 & treatment == 1) %>%  # 过滤出处理权重小于0且处理变量为1的行
  nrow()  # 计算符合条件的行数

# 输出初级教育数据中负权重的处理观察值数量
n_treated_negative_primary

# 计算负权重的处理观察值占总处理人数的比例
n_treated_negative_primary / n_treated_primary

# 计算中级教育数据中负权重的处理观察值数量
n_treated_negative_secondary <- fpe_secondary_weights %>%
  filter(treatment_weight < 0 & treatment == 1) %>%  # 过滤出处理权重小于0且处理变量为1的行
  nrow()  # 计算符合条件的行数

# 输出中级教育数据中负权重的处理观察值数量
n_treated_negative_secondary

# 计算负权重的处理观察值占总处理人数的比例
n_treated_negative_secondary / n_treated_secondary
```

50

0.259067357512953

36

0.260869565217391

``` r
# 创建初级教育的权重数据框，并将处理变量转换为有序因子
plot_weights_primary <- fpe_primary_weights %>%
  mutate(treatment_fct = factor(treatment, labels = c("Untreated", "Treated"), ordered = TRUE)) %>%  # 将处理变量转换为因子，标记为“未处理”和“已处理”
  mutate(oh_no = (treatment == 1 & treatment_weight < 0) | (treatment == 0 & treatment_weight > 0))  # 标记处理权重异常的观察值

# 创建中级教育的权重数据框，并将处理变量转换为有序因子
plot_weights_secondary <- fpe_secondary_weights %>%
  mutate(treatment_fct = factor(treatment, labels = c("Untreated", "Treated"), ordered = TRUE)) %>%  # 将处理变量转换为因子，标记为“未处理”和“已处理”
  mutate(oh_no = (treatment == 1 & treatment_weight < 0) | (treatment == 0 & treatment_weight > 0))  # 标记处理权重异常的观察值

# 绘制初级教育的处理权重直方图
hist_primary <- ggplot(plot_weights_primary,
                       aes(x = treatment_weight, fill = treatment_fct, alpha = oh_no)) +  # 设置x轴为处理权重，填充颜色为处理因子，透明度为异常标记
  geom_histogram(binwidth = 0.002, color = "white", boundary = 0,
                 position = position_identity()) +  # 绘制直方图，设置边界和颜色
  geom_vline(xintercept = 0, color = "#FF4136", size = 0.5) +  # 添加垂直线表示权重为0的位置
  scale_alpha_manual(values = c(0.6, 1)) +  # 设置透明度的手动映射
  scale_fill_viridis_d(option = "rocket", begin = 0.2, end = 0.8) +  # 设置填充颜色的手动映射
  labs(x = "Residualized treatment", y = "Count",  # 设置x轴和y轴标签
       title = "Primary school enrollment") +  # 设置图表标题
  guides(fill = "none", alpha = "none") +  # 不显示图例
  facet_wrap(vars(treatment_fct), ncol = 1) +  # 按处理因子分面
  theme_clean()  # 应用干净的主题

# 绘制中级教育的处理权重直方图
hist_secondary <- ggplot(plot_weights_secondary,
                         aes(x = treatment_weight, fill = treatment_fct, alpha = oh_no)) +  # 设置x轴为处理权重，填充颜色为处理因子，透明度为异常标记
  geom_histogram(binwidth = 0.002, color = "white", boundary = 0,
                 position = position_identity()) +  # 绘制直方图，设置边界和颜色
  geom_vline(xintercept = 0, color = "#FF4136", size = 0.5) +  # 添加垂直线表示权重为0的位置
  scale_alpha_manual(values = c(0.6, 1)) +  # 设置透明度的手动映射
  scale_fill_viridis_d(option = "rocket", begin = 0.2, end = 0.8) +  # 设置填充颜色的手动映射
  labs(x = "Residualized treatment", y = "Count",  # 设置x轴和y轴标签
       title = "Secondary school enrollment") +  # 设置图表标题
  guides(fill = "none", alpha = "none") +  # 不显示图例
  facet_wrap(vars(treatment_fct), ncol = 1) +  # 按处理因子分面
  theme_clean()  # 应用干净的主题

# 将初级教育和中级教育的直方图合并显示
hist_primary + hist_secondary
```

<figure>
<img
src="http://www.plutoese.com:8888/images/2025/05/20/output_65_0.png"
alt="png" />
<figcaption aria-hidden="true">png</figcaption>
</figure>

哪些接受处理的国家-年份获得了这些负权重呢？根据雅基耶拉的说法，早期采用者和较晚的年份更有可能发生这种转变。

``` r
# 创建初级教育的权重数据框，并根据处理残差和处理状态标记观察值的类型
plot_waffle_primary <- fpe_primary_weights %>%
  mutate(weight_fill = case_when(
    treatment_resid < 0 & treatment ~ "Treatment observations, negative weight",  # 处理残差为负且为处理组的观察值
    treatment_resid > 0 & treatment ~ "Treatment observations, positive weight",  # 处理残差为正且为处理组的观察值
    !treatment ~ "Comparison observations"  # 非处理组的观察值
  )) %>%
  arrange(desc(fpe_year), desc(country)) %>%  # 按年份和国家降序排列
  mutate(country = fct_inorder(country)) %>%  # 将国家变量转换为有序因子
  mutate(weight_fill = fct_inorder(weight_fill))  # 将权重填充变量转换为有序因子

# 创建中级教育的权重数据框，并根据处理残差和处理状态标记观察值的类型
plot_waffle_secondary <- fpe_secondary_weights %>%
  mutate(weight_fill = case_when(
    treatment_resid < 0 & treatment ~ "Treatment observations, negative weight",  # 处理残差为负且为处理组的观察值
    treatment_resid > 0 & treatment ~ "Treatment observations, positive weight",  # 处理残差为正且为处理组的观察值
    !treatment ~ "Comparison observations"  # 非处理组的观察值
  )) %>%
  arrange(desc(fpe_year), desc(country)) %>%  # 按年份和国家降序排列
  mutate(country = fct_inorder(country)) %>%  # 将国家变量转换为有序因子
  mutate(weight_fill = fct_inorder(weight_fill))  # 将权重填充变量转换为有序因子

# 绘制初级教育的瓦片图
waffle_primary <- ggplot(plot_waffle_primary,
                         aes(x = year, y = country, fill = weight_fill)) +  # 设置x轴为年份，y轴为国家，填充颜色为权重填充
  geom_tile(color = "white", size = 0.5) +  # 绘制瓦片，设置边框颜色和大小
  scale_fill_manual(values = c("grey50", "#0074D9", "#FF4136"),  # 设置填充颜色
                    guide = guide_legend(reverse = TRUE),  # 图例反向显示
                    name = NULL) +  # 不显示图例标题
  scale_x_continuous(expand = expansion(add = 0.5),  # 设置x轴扩展
                     breaks = seq(1980, 2015, 5)) +  # 设置x轴刻度
  labs(x = NULL, y = NULL, title = "Primary school enrollment") +  # 设置x轴和y轴标签及标题
  coord_equal() +  # 设置坐标轴比例相等
  theme_clean() +  # 应用干净的主题
  theme(legend.position = "bottom",  # 设置图例位置
        panel.grid = element_blank(),  # 不显示网格线
        legend.key.size = unit(0.8, "lines"))  # 设置图例键的大小

# 绘制中级教育的瓦片图
waffle_secondary <- ggplot(plot_waffle_secondary,
                         aes(x = year, y = country, fill = weight_fill)) +  # 设置x轴为年份，y轴为国家，填充颜色为权重填充
  geom_tile(color = "white", size = 0.5) +  # 绘制瓦片，设置边框颜色和大小
  scale_fill_manual(values = c("grey50", "#0074D9", "#FF4136"),  # 设置填充颜色
                    guide = guide_legend(reverse = TRUE),  # 图例反向显示
                    name = NULL) +  # 不显示图例标题
  scale_x_continuous(expand = expansion(add = 0.5),  # 设置x轴扩展
                     breaks = seq(1980, 2015, 5)) +  # 设置x轴刻度
  labs(x = NULL, y = NULL, title = "Secondary school enrollment") +  # 设置x轴和y轴标签及标题
  coord_equal() +  # 设置坐标轴比例相等
  theme_clean() +  # 应用干净的主题
  theme(legend.position = "bottom",  # 设置图例位置
        panel.grid = element_blank(),  # 不显示网格线
        legend.key.size = unit(0.8, "lines"))  # 设置图例键的大小

# 合并初级教育和中级教育的瓦片图
waffle_primary / waffle_secondary
```

<figure>
<img
src="http://www.plutoese.com:8888/images/2025/05/20/output_67_0.png"
alt="png" />
<figcaption aria-hidden="true">png</figcaption>
</figure>

参考

- https://www.andrewheiss.com/blog/2021/08/25/twfe-diagnostics/

### DID包

<p align="center">
<img src="http://www.plutoese.com:8888/images/2025/05/20/did01.png" alt="描述文字" width=600 />
</p>
<p align="center">
<img src="http://www.plutoese.com:8888/images/2025/05/20/did02.png" alt="描述文字" />
</p>

**导入包**

``` r
# 加载必要的库
library("did")  # 用于因果推断的库
library("panelView")  # 用于面板数据可视化的库
```

    ## See bit.ly/panelview4r for more info.
    ## Report bugs -> yiqingxu@stanford.edu.

​

**生成模拟数据**

``` r
# 设置随机种子，以确保结果可重复
set.seed(12345)

# 数据生成过程：设置6个时间段（在采用后）和1000个观察值
dgp <- reset.sim(
                time.periods <- 6,  # 时间段数量
                n = 1000,  # 观察值数量
                ipw = TRUE,  # 启用逆概率加权
                reg = TRUE  # 启用回归
                )
dgp$te <- 0  # 初始化处理效应为0

# 添加动态效应，te.e表示每个时间段的效应
dgp$te.e <- 1:time.periods  # 动态效应从1到时间段数量

# 删除在第1期实施EBP的观察值。根据Callaway & Sant'Anna的说法，这些观察值对估计ATT(g, t)没有帮助。
data1 <- build_sim_dataset(dgp)  # 构建模拟数据集

# 生成时间上的采用指示器。如果观察值在采用时被编码为1，并且在之后的所有时间段也为1。
data1$long[data1$period < data1$G] = 0  # 在采用之前的时间段编码为0
data1$long[data1$period >= data1$G] = 1  # 在采用时及之后的时间段编码为1

# 计算在删除了在第1期采用EBP的观察值后，剩余的观察值数量
nrow(data1)  # 返回剩余观察值的数量

# 显示数据集的前几行
head(data1)  # 查看数据集的前几行
```

    Warning message:
    “Unknown or uninitialised column: `long`.”

5124

<table class="dataframe">
<caption>
A tibble: 6 × 8
</caption>
<thead>
<tr>
<th scope="col">
G
</th>
<th scope="col">
X
</th>
<th scope="col">
id
</th>
<th scope="col">
cluster
</th>
<th scope="col">
period
</th>
<th scope="col">
Y
</th>
<th scope="col">
treat
</th>
<th scope="col">
long
</th>
</tr>
<tr>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<int\>
</th>
<th scope="col">
\<int\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
<th scope="col">
\<dbl\>
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
2
</td>
<td>
0.709466
</td>
<td>
1
</td>
<td>
21
</td>
<td>
1
</td>
<td>
1.076000
</td>
<td>
1
</td>
<td>
0
</td>
</tr>
<tr>
<td>
2
</td>
<td>
0.709466
</td>
<td>
1
</td>
<td>
21
</td>
<td>
2
</td>
<td>
4.417041
</td>
<td>
1
</td>
<td>
1
</td>
</tr>
<tr>
<td>
2
</td>
<td>
0.709466
</td>
<td>
1
</td>
<td>
21
</td>
<td>
3
</td>
<td>
6.040249
</td>
<td>
1
</td>
<td>
1
</td>
</tr>
<tr>
<td>
2
</td>
<td>
0.709466
</td>
<td>
1
</td>
<td>
21
</td>
<td>
4
</td>
<td>
8.204727
</td>
<td>
1
</td>
<td>
1
</td>
</tr>
<tr>
<td>
2
</td>
<td>
0.709466
</td>
<td>
1
</td>
<td>
21
</td>
<td>
5
</td>
<td>
14.120563
</td>
<td>
1
</td>
<td>
1
</td>
</tr>
<tr>
<td>
2
</td>
<td>
0.709466
</td>
<td>
1
</td>
<td>
21
</td>
<td>
6
</td>
<td>
13.713790
</td>
<td>
1
</td>
<td>
1
</td>
</tr>
</tbody>
</table>

**可视化数据**

``` r
# 使用panelview函数可视化面板数据
panelview(
    data = data1,  # 输入的数据集
    formula = Y ~ long,  # 指定模型公式，Y为因变量，long为自变量
    index = c("id", "period"),  # 指定面板数据的索引，id为个体标识，period为时间段
    xlab = "Year",  # x轴标签设置为“Year”
    ylab = "Unit",  # y轴标签设置为“Unit”
    gridOff = TRUE,  # 关闭网格线
    by.timing = TRUE,  # 按时间段分组显示
    pre.post = FALSE,  # 不显示采用前后的对比
    random.select = FALSE  # 不随机选择观察值
)
```

<figure>
<img
src="http://www.plutoese.com:8888/images/2025/05/20/output_75_0.png"
alt="png" />
<figcaption aria-hidden="true">png</figcaption>
</figure>

使用att_gt函数来估计处理组的组 -
时间平均处理效应（ATT）。此输出包含组与时间的组合情况。例如，当在组别 =
4 和时间周期 = 5 实施循证实践（EBP）时，该组的 ATT 为 2.0280（95%
置信区间：1.4894，2.5667） 。

``` r
# 使用att_gt函数计算处理效应
att_grouptime <- att_gt(yname = "Y",  # 指定因变量Y
                        tname = "period",  # 指定时间变量period
                        idname = "id",  # 指定个体标识变量id
                        gname = "G",  # 指定采用组变量G
                        xformla = ~ X,  # 指定控制变量的公式，这里为X
                        data = data1  # 输入的数据集
)

# 总结处理效应的结果
summary(att_grouptime)  # 显示att_grouptime对象的摘要信息
```

    Call:
    att_gt(yname = "Y", tname = "period", idname = "id", gname = "G", 
        xformla = ~X, data = data1)

    Reference: Callaway, Brantly and Pedro H.C. Sant'Anna.  "Difference-in-Differences with Multiple Time Periods." Journal of Econometrics, Vol. 225, No. 2, pp. 200-230, 2021. <https://doi.org/10.1016/j.jeconom.2020.12.001>, <https://arxiv.org/abs/1803.09015> 

    Group-Time Average Treatment Effects:
     Group Time ATT(g,t) Std. Error [95% Simult.  Conf. Band]  
         2    2   1.0889     0.1627        0.5965      1.5813 *
         2    3   1.7708     0.1690        1.2595      2.2822 *
         2    4   2.8899     0.1659        2.3877      3.3921 *
         2    5   3.8359     0.1674        3.3293      4.3425 *
         2    6   5.0540     0.1707        4.5372      5.5707 *
         3    2   0.1012     0.1651       -0.3986      0.6010  
         3    3   0.8082     0.1702        0.2931      1.3233 *
         3    4   1.8602     0.1659        1.3580      2.3625 *
         3    5   2.8019     0.1578        2.3242      3.2795 *
         3    6   4.0831     0.1684        3.5734      4.5928 *
         4    2  -0.0436     0.1733       -0.5682      0.4810  
         4    3  -0.0378     0.1819       -0.5884      0.5128  
         4    4   0.9011     0.1785        0.3610      1.4412 *
         4    5   2.0280     0.1650        1.5286      2.5275 *
         4    6   3.1085     0.1936        2.5226      3.6943 *
         5    2   0.1178     0.1694       -0.3949      0.6305  
         5    3  -0.2093     0.1896       -0.7831      0.3644  
         5    4  -0.0799     0.1987       -0.6812      0.5214  
         5    5   1.3767     0.1704        0.8609      1.8925 *
         5    6   2.4207     0.1848        1.8614      2.9801 *
         6    2   0.0663     0.1761       -0.4668      0.5995  
         6    3  -0.2224     0.1833       -0.7773      0.3325  
         6    4  -0.0099     0.1815       -0.5593      0.5394  
         6    5   0.2411     0.1994       -0.3625      0.8447  
         6    6   0.9987     0.1766        0.4642      1.5332 *
    ---
    Signif. codes: `*' confidence band does not cover 0

    P-value for pre-test of parallel trends assumption:  0.88028
    Control Group:  Never Treated,  Anticipation Periods:  0
    Estimation Method:  Doubly Robust

我们可以绘制出组 -
时间平均处理效应的结果。ggdid函数会根据循证实践（EBP）实施的时间来绘制每个组的情况。在这个例子中，循证实践（EBP）有
5 个可能的实施时间点（时间周期 = 2、3、4、5 和
6）。请记住，我们剔除了在时间周期 1
实施循证实践（EBP）的情况，因为那些观测值对平均处理效应（ATT）没有贡献。

``` r
# 使用ggdid函数可视化处理效应的结果
# att_grouptime是通过att_gt函数计算得到的处理效应对象
# ylim参数设置y轴的范围为-1到6，以便更好地展示处理效应的变化
ggdid(att_grouptime, ylim = c(-1, 6))  
```

<figure>
<img
src="http://www.plutoese.com:8888/images/2025/05/20/output_79_0.png"
alt="png" />
<figcaption aria-hidden="true">png</figcaption>
</figure>

**卡拉韦（Callaway）和桑特安娜（Sant’Anna）的双重差分（DID）估计量**

我们希望将所有这些不同的子实验汇总为一个单一的加权平均处理效应（ATT）。为此，我们需要使用aggte函数。总的来说，当循证实践（EBP）实施时，我们有
5 个这样的组（时间周期为 2、3、4、5 和
6）。此外，aggte函数将根据组（实施发生的组）和时间，用权重w(g,t)对这些组进行加权。

``` r
# 使用att_gt函数计算处理效应，返回一个包含处理效应估计的对象
att_grouptime <- att_gt(yname = "Y",  # 指定因变量Y
                        tname = "period",  # 指定时间变量period
                        idname = "id",  # 指定个体标识变量id
                        gname = "G",  # 指定采用组变量G
                        xformla = ~ X,  # 指定控制变量的公式，这里为X
                        data = data1  # 输入的数据集
                        )

# 聚合不同实施时间段的ATT（平均处理效应），以便分析不同组的处理效应
att_aggregate <- aggte(att_grouptime, type = "dynamic")  # 计算动态处理效应的聚合结果

# 显示聚合处理效应的摘要信息，便于理解处理效应的整体趋势
summary(att_aggregate)  # 输出att_aggregate对象的摘要信息
```

    Call:
    aggte(MP = att_grouptime, type = "dynamic")

    Reference: Callaway, Brantly and Pedro H.C. Sant'Anna.  "Difference-in-Differences with Multiple Time Periods." Journal of Econometrics, Vol. 225, No. 2, pp. 200-230, 2021. <https://doi.org/10.1016/j.jeconom.2020.12.001>, <https://arxiv.org/abs/1803.09015> 

​  
​Overall summary of ATT’s based on event-study/dynamic aggregation:  
​ATT Std. Error \[ 95% Conf. Int.\]  
​2.9992 0.0859 2.8307 3.1676 \*

​  
​Dynamic Effects: ​ Event time Estimate Std. Error \[95% Simult. Conf.
Band\]  
​-4 0.0663 0.1637 -0.3895 0.5222  
​-3 -0.0523 0.1099 -0.3585 0.2539  
​-2 -0.0883 0.0779 -0.3052 0.1285  
​-1 0.0557 0.0729 -0.1474 0.2588  
​0 1.0436 0.0608 0.8743 1.2130 * ​ 1 2.0189 0.0798 1.7968 2.2410 * ​ 2
2.9352 0.0970 2.6652 3.2052 * ​ 3 3.9441 0.1192 3.6121 4.2761 * ​ 4 5.0540
0.1786 4.5565 5.5515 * ​ — ​ Signif. codes: \`*’ confidence band does not
cover 0 ​  
Control Group: Never Treated, Anticipation Periods: 0 Estimation Method:
Doubly Robust

基于交错双重差分（cs staggered
DID）方法，汇总后的总体平均处理效应（ATT）为 2.9992（95%
置信区间：2.8277，3.1706）。

可以使用ggdid函数绘制交错双重差分（cs staggered
DID）方法的结果。横轴表示接触（干预）的时长，即自循证实践（EBP）实施以来的时间。在时间周期
= 0 时，这是观测对象实施循证实践（EBP）的时间点。在时间周期 = -1
时，这是循证实践（EBP）实施之前的时间段。
从直观上看，平行趋势似乎成立，因为在实施前时间段的点估计值处于或接近零，且在统计上不显著。总体而言，循证实践（EBP）实施后的时间段内的平均处理效应（ATT）为正，并且在统计上显著。

``` r
# 使用ggdid函数可视化聚合处理效应的结果
# att_aggregate是通过aggte函数计算得到的聚合处理效应对象
# 该函数将展示不同时间段的平均处理效应
ggdid(att_aggregate) 
```

<figure>
<img
src="http://www.plutoese.com:8888/images/2025/05/20/output_83_0.png"
alt="png" />
<figcaption aria-hidden="true">png</figcaption>
</figure>

注意：比较组是 “从未接受过处理的组”。不过，人们也可以将
“尚未接受处理的组” 用作对照组，这一对照组将包含 “从未接受过处理的组” 和
“尚未接受处理的组” 这两类。这可以通过使用 “control_group” 选项来实现。

``` r
# 计算未处理组的处理效应
# 使用att_gt函数计算未处理组的处理效应，返回一个包含处理效应估计的对象
att_grouptime_notyet <- att_gt(yname = "Y",  # 指定因变量Y
                               tname = "period",  # 指定时间变量period
                               idname = "id",  # 指定个体标识变量id
                               gname = "G",  # 指定采用组变量G
                               xformla = ~ X,  # 指定控制变量的公式，这里为X
                               control_group = "notyettreated",  # 指定未处理组
                               data = data1  # 输入的数据集
)

# 聚合不同实施时间段的ATT（平均处理效应），以便分析未处理组的处理效应
att_aggregate_notyet <- aggte(att_grouptime_notyet, type = "dynamic")  # 计算动态处理效应的聚合结果

# 显示聚合处理效应的摘要信息，便于理解未处理组的处理效应整体趋势
summary(att_aggregate_notyet)  # 输出att_aggregate_notyet对象的摘要信息
```

    Call:
    aggte(MP = att_grouptime_notyet, type = "dynamic")

    Reference: Callaway, Brantly and Pedro H.C. Sant'Anna.  "Difference-in-Differences with Multiple Time Periods." Journal of Econometrics, Vol. 225, No. 2, pp. 200-230, 2021. <https://doi.org/10.1016/j.jeconom.2020.12.001>, <https://arxiv.org/abs/1803.09015> 

​  
​Overall summary of ATT’s based on event-study/dynamic aggregation:  
​ATT Std. Error \[ 95% Conf. Int.\]  
​3.0041 0.0755 2.8561 3.1522 \*

​  
​Dynamic Effects: ​ Event time Estimate Std. Error \[95% Simult. Conf.
Band\]  
​-4 -0.0402 0.1192 -0.3723 0.2918  
​-3 0.0131 0.1020 -0.2709 0.2970  
​-2 -0.0698 0.0878 -0.3144 0.1747  
​-1 0.0793 0.0791 -0.1409 0.2995  
​0 1.0302 0.0634 0.8536 1.2067 * ​ 1 2.0456 0.0694 1.8522 2.2389 * ​ 2
2.9693 0.0907 2.7166 3.2220 * ​ 3 3.9216 0.1058 3.6269 4.2163 * ​ 4 5.0540
0.1833 4.5434 5.5645 * ​ — ​ Signif. codes: \`*’ confidence band does not
cover 0 ​  
Control Group: Not Yet Treated, Anticipation Periods: 0 Estimation
Method: Doubly Robust

``` r
# 使用ggdid函数可视化聚合处理效应的结果
# att_aggregate_notyet是通过aggte函数计算得到的聚合处理效应对象
# 该函数将展示不同时间段的平均处理效应
ggdid(att_aggregate_notyet) 
```

​  
![png](http://www.plutoese.com:8888/images/2025/05/20/output_86_0.png) ​

参考

- https://rpubs.com/mbounthavong/cs_staggered_did_r
- https://www.lianxh.cn/news/762e878e7063b.html

``` r
```